# 프로젝트 : 얼굴을 인식하여 캐릭터 씌우기

## Face Detection vs Face Recognition

In [5]:
import cv2
import mediapipe as mp

def overlay(image, x, y, w, h, overlay_image): # 대상 이미지(3채널), x, y 좌표,  width, height, 덮어씌울 이미지 (4채널)
  alpha = overlay_image[:, :, 3] # BGRA -> 같은 x, y좌표값에 B, G, R, A 4개의 값이 저장되어 있음. 그중 alpha값
  mask_image = alpha / 255 # 0 ~255 -> 0 ~ 1 사이의 값 (1: 불투명, 0: 투명)

  for c in range(0, 3): # channel BGR
    image[y-h:y+h, x-w:x+w, c] = (overlay_image[:, :, c] * mask_image) + (image[y-h:y+h, x-w:x+w, c] * (1 - mask_image)) # alpha값의 투명도를 곱한 4채널의 BGR값 + 대상이미지의 3채널 BGR값에 (1-alpha)값을 곱한 값 -> 결과적으로 불투명한 이미지가 되지만 왜 alpha값을 쓰냐?
    # -> 추가적으로 넣을 이미지에 alpha값이 0인 배경 부분을 제거하고 필요한 부분만 넣고 원본 이미지(동영상)를 실행 시키기 위해 alpha값을 사용하는듯?

# 얼굴을 찾고, 찾은 얼굴에 표시를 해주기 위한 변수 정의
mp_face_detection = mp.solutions.face_detection # 얼굴 검출을 위한 face_detection 모듈 사용
mp_drawing = mp.solutions.drawing_utils # 얼굴 특징을 그리기 위한 drawing_utils 모듈 사용

# For video input:
cap = cv2.VideoCapture('face.mp4')

# 이미지 불러오기
image_right_eye = cv2.imread('right_eye.png', cv2.IMREAD_UNCHANGED) # 100 x 100, alpha값까지 추가된 정보 불러옴.
image_left_eye = cv2.imread('left_eye.png', cv2.IMREAD_UNCHANGED) # 100 x 100, alpha값까지 추가된 정보 불러옴.
image_nose = cv2.imread('nose.png', cv2.IMREAD_UNCHANGED) # 300 x 100, alpha값까지 추가된 정보 불러옴.


with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.7) as face_detection: # model_selection : 0일경우 2m이내, 1일 경우 2m 밖 / min_detection_confidence : 어느 정도의 신뢰도로 얼굴을 인식할건지
  while cap.isOpened():
      success, image = cap.read()
      if not success:
        # If loading a video, use 'break' instead of 'continue'.
        break

      # To improve performance, optionally mark the image as not writeable to
      # pass by reference.
      image.flags.writeable = False
      image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      results = face_detection.process(image)

      # Draw the face detection annotations on the image.
      image.flags.writeable = True
      image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

      if results.detections:
        # 6개의 특징 : 오른쪽 눈, 왼쪽 눈, 코 끝부분, 입 중심, 오른쪽 귀, 왼쪽 귀(귀 구슬점)
        for detection in results.detections:
          # mp_drawing.draw_detection(image, detection)
          # print(detection)

          # 특정 위치 가져오기
          keypoints = detection.location_data.relative_keypoints
          right_eye = keypoints[0]
          left_eye = keypoints[1]
          nose_tip = keypoints[2]

          h, w, _ = image.shape # height, width, channel
          right_eye = (int(right_eye.x * w) - 20, int(right_eye.y * h) - 100)
          left_eye = (int(left_eye.x * w)+ 20, int(left_eye.y * h) - 100)
          nose_tip = (int(nose_tip.x * w), int(nose_tip.y * h))

          # 양 눈, 코에 동그라미 그리기
          # cv2.circle(image, right_eye, 50, (255, 0, 0), 10, cv2.LINE_AA)
          # cv2.circle(image, left_eye, 50, (0, 255, 0), 10, cv2.LINE_AA)
          # cv2.circle(image, nose_tip, 50, (0, 255, 255), 10, cv2.LINE_AA)

          # 이미지 넣기
          
          # image[(right_eye[1] - int(image_right_eye.shape[0] / 2)) : (right_eye[1] + int(image_right_eye.shape[0] / 2)), (right_eye[0] - int(image_right_eye.shape[1] / 2)) : (right_eye[0] + int(image_right_eye.shape[1] / 2))] = image_right_eye
          # image[(left_eye[1] - int(image_left_eye.shape[0] / 2)) : (left_eye[1] + int(image_left_eye.shape[0] / 2)), (left_eye[0] - int(image_left_eye.shape[1] / 2)) : (left_eye[0] + int(image_left_eye.shape[1] / 2))] = image_left_eye
          # image[(nose_tip[1] - int(image_nose.shape[0] / 2)) : (nose_tip[1] + int(image_nose.shape[0] / 2)), (nose_tip[0] - int(image_nose.shape[1] / 2)) : (nose_tip[0] + int(image_nose.shape[1] / 2))] = image_nose

          # image, x, y, w, h, overlay_image
          overlay(image, *right_eye, int(image_right_eye.shape[1] / 2), int(image_right_eye.shape[0] / 2), image_right_eye)
          overlay(image, *left_eye, int(image_left_eye.shape[1] / 2 ), int(image_left_eye.shape[0] / 2), image_left_eye)
          overlay(image, *nose_tip, int(image_nose.shape[1] / 2), int(image_nose.shape[0] / 2), image_nose)

      # Flip the image horizontally for a selfie-view display.
      cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None, fx=0.5, fy=0.5))

      if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [4]:
image[:, :, 1]

array([[ 26,  26,  24, ...,  97,  97,  98],
       [ 26,  26,  24, ...,  97,  97,  98],
       [ 26,  26,  24, ...,  97,  97,  98],
       ...,
       [139, 139, 138, ..., 129, 129, 129],
       [139, 138, 139, ..., 128, 129, 129],
       [139, 139, 139, ..., 128, 128, 129]], dtype=uint8)

image_right_eye = cv2.imread('right_eye.png', cv2.IMREAD_UNCHANGED)
image_right_eye[:, :, 3]

# 자세한 공부
> python opencv readthedocs 검색